In [1]:
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()


In [2]:
len(text)

1115393

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [25]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
10


In [13]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [17]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)


In [18]:
n = int(0.9 * len(text))
train_data = data[:n]
val_data = data[n:]

In [19]:
torch.manual_seed(1337)

block_size = 8
batch_size = 4
train_data[:block_size + 1]

def get_batch(split):
    data = train_data if split == "train" else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

xb, yb = get_batch("train")

In [43]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            # we're just predicting single token
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # note no targets
            logits, loss = self(idx)
            # just pluck out the last prediction in T dimension
            # (B, T, C) -> (B, C)
            logits = logits[:, -1, :]

            # calculate softmax for the results for the inputs
            probs = F.softmax(logits, dim=-1)

            # 1 sample
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # becomes (, T+1)
            idx = torch.cat((idx, idx_next), dim=-1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

NEWLINE_IDX = 0
idx = torch.tensor(([[NEWLINE_IDX]]), dtype=torch.long)

output = m.generate(idx, max_new_tokens=100)[0] # because first batch
output = output.tolist() # convert from tensor to list
output = decode(output) # to string
print(output)



torch.Size([32, 65])
tensor(4.9407, grad_fn=<NllLossBackward0>)

Cv Mx:3SYCS.Tb
lGaNuaJ;:?T-X$lgwlqFkAxR&oXzQjjq?EaBWhP;mVTMAcxv;v?qW,:GHVMvfeXiC?,!KBzRfvK
!EVA?nATT


In [44]:
# training the bigram model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [57]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.2935690879821777


In [83]:
# try generating again
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


pllay hevelo wom st ow ur,
Andsees, s ourit maiqu.
aces it nty his or houdo nde bus thove wint-PEMENCld H:
MENIS: the t 't de.
WI y hreontangr iceesquteruawe, rd f h y m, her d acth coreco
ARCLANERCKI mbel ghathol ll t s t athen KE:
Th. woQUCO thiroulth

mesc
Tes pstar, ounlearandyorcickeaimuf pinth
